In [53]:
import pandas as pd
import numpy as np

# Preparacion de datos

In [57]:
def get_postulaciones(size=None):
    postulaciones = pd.read_csv('data/FiubaHasta15Abril/fiuba_4_postulaciones.csv', nrows =size).drop_duplicates(subset=['idpostulante', 'idaviso'], keep='last')
    columns_rename = {'idaviso': 'id_aviso', 'idpostulante': 'id_postulante', 'fechapostulacion': 'fecha_postulacion'}
    postulaciones = postulaciones.rename(columns=columns_rename)
    postulaciones['fecha_postulacion']=pd.to_datetime(postulaciones['fecha_postulacion'])
    return postulaciones

#Devuelve las vistas indexadas por un hashtag que es id_aviso + id_postulante
def get_vistas(size=None):
    vistas = pd.read_csv('data/FiubaHasta15Abril/fiuba_3_vistas.csv', nrows =size)
    #vistas['hash'] = vistas['idpostulante'] + vistas['idAviso'].astype(str)    
    vistas_sumarizadas = vistas.groupby(['idpostulante', 'idAviso'], as_index=False)['timestamp'].count()
    columns_rename = {'timestamp': 'visualizaciones', 'idpostulante':'id_postulante', 'idAviso':'id_aviso'}
    vistas_sumarizadas = vistas_sumarizadas.rename(columns=columns_rename)    
    return vistas_sumarizadas

def get_avisos_detalle():
    avisos1 = pd.read_csv('data/fiuba_6_avisos_detalle.csv')
    avisos2 = pd.read_csv('data/FiubaDesde15Abril/fiuba_6_avisos_detalle.csv')
    avisos3 = pd.read_csv('data/FiubaHasta15Abril/fiuba_6_avisos_detalle.csv')
    avisos4 = pd.read_csv('data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv')        
    avisos_detalle = pd.concat([avisos1,avisos2,avisos3,avisos4]).drop_duplicates(subset=['idaviso'], keep='last').reset_index(drop=True)
    columns_rename = {'idpostulante': 'id_postulante', 'idaviso': 'id_aviso'}
    avisos_detalle = avisos_detalle.rename(columns=columns_rename)
    to_nivel_laboral_nro = {'Senior / Semi-Senior' : 2, 'Junior':1, 'Otro':0,
       'Jefe / Supervisor / Responsable':3,
       'Gerencia / Alta Gerencia / Dirección':4}
    to_tipo_trabajo_nro={'Full-time':0, 'Part-time':1, 'Teletrabajo':2, 'Por Horas':3, 'Pasantia':4,
       'Temporario':5, 'Por Contrato':6, 'Fines de Semana':7, 'Primer empleo':8,
       'Voluntario':9}
    to_nombre_area_numero = pd.Series(avisos_detalle['nombre_area'].unique()).to_dict()
    to_nombre_area_numero  = {v: k for k, v in to_nombre_area_numero.items()}
    avisos_detalle['nivel_laboral']= avisos_detalle['nivel_laboral'].map(to_nivel_laboral_nro)
    avisos_detalle['tipo_de_trabajo'] = avisos_detalle['tipo_de_trabajo'].map(to_tipo_trabajo_nro)
    avisos_detalle['nombre_area'] = avisos_detalle['nombre_area'].map(to_nombre_area_numero)
    
    return avisos_detalle

def get_year_of_birth(postulantes_genero_edad):
    return (pd.to_datetime
            (postulantes_genero_edad['fechanacimiento'], errors='coerce', format='%Y-%m-%d')
            .dt.year)

def get_age(yearOfBirth):
    return 2018 - yearOfBirth
    
def get_age_range(yearOfBirth):
    age = get_age(yearOfBirth)
    if(age<25): return 'Entre 18 y 24'
    if(age<30): return 'Entre 25 y 30'
    if(age<35): return 'Entre 30 y 35'
    if(age<40): return 'Entre 35 y 40'
    if(age<45): return 'Entre 40 y 45'
    if(age<50): return 'Entre 45 y 50'
    return 'Mayor de 50'

def get_order_for_age_range():
    return ['Entre 18 y 24', 'Entre 25 y 30','Entre 30 y 35','Entre 35 y 40','Entre 40 y 45','Entre 45 y 50', 'Mayor de 50']

def get_postulantes_nivel_educativo_para(path):
    postulantes_nivel_educativo = pd.read_csv(path)
    columns_rename = {'idpostulante': 'id_postulante', 'nombre': 'formacion_postulante', 'estado': 'estado_formacion_postulante'}
    postulantes_nivel_educativo=postulantes_nivel_educativo.rename(columns=columns_rename)
    formacion_to_number={'Secundario' : 10, 'Otro': 20, 'Terciario/Técnico' : 30, 'Universitario' : 40, 'Posgrado' : 50,
    'Master' : 50, 'Doctorado' : 50}
    postulantes_nivel_educativo['formacion_postulante_numero']=postulantes_nivel_educativo['formacion_postulante'].map(formacion_to_number);
    estado_to_number = {'En Curso': 4, 'Abandonado': 0, 'Graduado': 8}
    postulantes_nivel_educativo['estado_formacion_postulante_numero']=postulantes_nivel_educativo['estado_formacion_postulante'].map(estado_to_number)
    postulantes_nivel_educativo['nivel_educativo_postulante_numero'] = postulantes_nivel_educativo['formacion_postulante_numero'] + postulantes_nivel_educativo['estado_formacion_postulante_numero']
    postulantes_nivel_educativo['nivel_educativo_postulante_texto'] = postulantes_nivel_educativo['formacion_postulante'] + ' - ' + postulantes_nivel_educativo['estado_formacion_postulante']
    relevant_columns = ['id_postulante','nivel_educativo_postulante_texto', 'nivel_educativo_postulante_numero']
    postulantes_nivel_educativo = postulantes_nivel_educativo[relevant_columns]
    grouped=postulantes_nivel_educativo.groupby(['id_postulante']).agg({'nivel_educativo_postulante_numero':['max']}) 
    df=grouped.reset_index()
    df.columns = ['id_postulante', 'maximo_nivel_educativo_postulante']
    return df

def get_postulantes_nivel_educativo():
    postulantes1 = get_postulantes_nivel_educativo_para('data/fiuba_1_postulantes_educacion.csv')
    postulantes2 = get_postulantes_nivel_educativo_para('data/FiubaDesde15Abril/fiuba_1_postulantes_educacion.csv')
    postulantes3 = get_postulantes_nivel_educativo_para('data/FiubaHasta15Abril/fiuba_1_postulantes_educacion.csv')
    return pd.concat([postulantes1,postulantes2,postulantes3]).drop_duplicates(subset=['id_postulante'], keep='last').reset_index(drop=True)

def get_postulantes_genero_edad():
    postulantes1 = pd.read_csv('data/fiuba_2_postulantes_genero_y_edad.csv')
    postulantes2 = pd.read_csv('data/FiubaDesde15Abril/fiuba_2_postulantes_genero_y_edad.csv')
    postulantes3 = pd.read_csv('data/FiubaHasta15Abril/fiuba_2_postulantes_genero_y_edad.csv')
    postulantes_genero_edad = pd.concat([postulantes1,postulantes2,postulantes3]).drop_duplicates(subset=['idpostulante'], keep='last').reset_index(drop=True)
    postulantes_genero_edad['año_nacimiento_postulante']=get_year_of_birth(postulantes_genero_edad)
    postulantes_genero_edad['edad_postulante']=postulantes_genero_edad['año_nacimiento_postulante'].map(get_age, na_action=None)
    postulantes_genero_edad['rango_edad_postulante']=postulantes_genero_edad['año_nacimiento_postulante'].map(get_age_range, na_action=None)
    columns_rename = {'idpostulante': 'id_postulante', 'fechanacimiento': 'fecha_nacimiento_postulante', 'sexo': 'genero_postulante'}
    postulantes_genero_edad = postulantes_genero_edad.rename(columns=columns_rename)
    postulantes_genero_edad = postulantes_genero_edad[['id_postulante', 'genero_postulante', 'fecha_nacimiento_postulante', 'edad_postulante', 'rango_edad_postulante']]
    postulantes_genero_edad['genero_postulante'] = postulantes_genero_edad['genero_postulante'].map({'FEM': 0, 'MASC': 1, 'NO_DECLARA': 2})
    return postulantes_genero_edad

def get_postulantes_limpios():
    postulantes = pd.merge(get_postulantes_genero_edad(), get_postulantes_nivel_educativo(), on='id_postulante', how='outer')
    order_for_columns = ['id_postulante','edad_postulante', 'genero_postulante', 'maximo_nivel_educativo_postulante']
    return postulantes[order_for_columns]

def get_detalle_postulaciones(size=None):
    postulaciones = get_postulaciones(size)
    avisos = get_avisos_detalle()
    postulantes = get_postulantes_limpios()
    detalle_postulaciones = pd.merge(postulantes, postulaciones, on='id_postulante', how='inner') 
    detalle_postulaciones = pd.merge(detalle_postulaciones, avisos, on='id_aviso', how='inner')
    
    return detalle_postulaciones

    

def get_detalle_vistas(size=None):
    vistas = get_vistas(size)
    avisos = get_avisos_detalle()
    postulantes = get_postulantes_limpios()
    detalle_vistas = pd.merge(postulantes, vistas, on='id_postulante', how='inner') 
    detalle_vistas = pd.merge(detalle_vistas, avisos, on='id_aviso', how='inner')
    return detalle_vistas

def x_entrenamiento():
    return ['edad_postulante', 'genero_postulante', 'maximo_nivel_educativo_postulante', 'nivel_laboral', 'titulo']

def y_entrenamiento():
    return ['sepostulo']

def columnas_relevantes_test_data():
    return ['id','id_aviso','id_postulante'] + x_entrenamiento()

def get_test_data():
    tests = pd.read_csv('data/Test/test_final_100k.csv')
    columns_rename = {'idpostulante': 'id_postulante', 'idaviso': 'id_aviso'}
    tests = tests.rename(columns=columns_rename)
    tests = pd.merge(tests, get_avisos_detalle(), on='id_aviso', how='left')
    tests = pd.merge(tests, get_postulantes_limpios(), on='id_postulante', how='left')
    return tests[columnas_relevantes_test_data()]

def get_default_null_values(df):
    return {'edad_postulante':int(df['edad_postulante'].mean()), 'genero_postulante':int(0), 'maximo_nivel_educativo_postulante':int(df['maximo_nivel_educativo_postulante'].mean()), 'nivel_laboral':int(0), 'titulo':str(0)}


def get_test_data_clean():
    tests = get_test_data()
    tests = tests.fillna(value=get_default_null_values(tests))
    return tests

def get_datos_entrenamiento(size=None):
    postulaciones_aplicadas = get_detalle_postulaciones(size)

    
    columnas_relevantes = x_entrenamiento() + y_entrenamiento()
    postulaciones_aplicadas['sepostulo'] = True
    postulaciones_no_aplicadas = get_detalle_vistas(size)
    postulaciones_no_aplicadas['sepostulo'] = False
    postulaciones_totales = postulaciones_aplicadas.append(postulaciones_no_aplicadas).drop_duplicates(subset=['id_aviso', 'id_postulante'], keep='first')[columnas_relevantes].dropna()
    return postulaciones_totales


In [66]:
postulaciones_aplicadas = get_detalle_postulaciones(1000)
postulaciones_totales = pd.merge(postulaciones_aplicadas, get_vistas(), on=['id_aviso', 'id_postulante'], how='left')



In [67]:
postulaciones_totales.head(100)

,id_postulante,edad_postulante,genero_postulante,maximo_nivel_educativo_postulante,id_aviso,fecha_postulacion,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,visualizaciones
0,NjlD,37.0,0.0,48.0,1112248724,2018-01-19 07:39:16,1,Analista de CRM y Marketing Digital,<p>¡En LoJack estamos buscando un Analista de ...,Gran Buenos Aires,NaN,NaN,0,2.0,58,Lo Jack,NaN
1,ZaO5,40.0,0.0,40.0,1112286523,2018-01-24 15:07:39,1,Secretaria de CEO,"<p style=""""><strong><span style="""">Importante ...",Gran Buenos Aires,NaN,NaN,0,2.0,27,Constructora Sudamericana,NaN
2,YVO3q,53.0,0.0,48.0,1112286523,2018-01-25 09:37:55,1,Secretaria de CEO,"<p style=""""><strong><span style="""">Importante ...",Gran Buenos Aires,NaN,NaN,0,2.0,27,Constructora Sudamericana,NaN
3,ZPWlE,56.0,0.0,44.0,1112286523,2018-01-25 11:25:55,1,Secretaria de CEO,"<p style=""""><strong><span style="""">Importante ...",Gran Buenos Aires,NaN,NaN,0,2.0,27,Constructora Sudamericana,NaN
4,e4pe2,44.0,0.0,58.0,1112286523,2018-01-29 13:59:57,1,Secretaria de CEO,"<p style=""""><strong><span style="""">Importante ...",Gran Buenos Aires,NaN,NaN,0,2.0,27,Constructora Sudamericana,NaN
5,ZaO5,40.0,0.0,40.0,1112272060,2018-01-24 15:20:10,1,Recepcionista Concesionaria Ford de San Isidro,"<p><strong>Ford Motors Argentina, seleccionará...",Gran Buenos Aires,NaN,NaN,0,2.0,56,ABC CONSULTORES,NaN
6,ZaO5,40.0,0.0,40.0,1112288401,2018-01-26 08:37:04,1,Recepcionista / Asistente General,"<p><span lang=""ES"">Empresa constructora ubicad...",Gran Buenos Aires,NaN,NaN,0,2.0,56,Sedico S.A.,NaN
7,ZPWlE,56.0,0.0,44.0,1112288401,2018-01-25 11:30:41,1,Recepcionista / Asistente General,"<p><span lang=""ES"">Empresa constructora ubicad...",Gran Buenos Aires,NaN,NaN,0,2.0,56,Sedico S.A.,NaN
8,ZaO5,40.0,0.0,40.0,1112300563,2018-01-30 13:35:48,1,Secretaria de Gerencia,<p>ATENTO- Compañía multinacional que brinda s...,Gran Buenos Aires,NaN,NaN,0,2.0,27,Atento,NaN
9,YVO3q,53.0,0.0,48.0,1112300563,2018-02-20 10:50:14,1,Secretaria de Gerencia,<p>ATENTO- Compañía multinacional que brinda s...,Gran Buenos Aires,NaN,NaN,0,2.0,27,Atento,NaN


In [65]:
postulaciones_totales['visualizaciones'].value_counts()

1.0     147
2.0      72
4.0      28
3.0      13
5.0       5
8.0       3
7.0       2
6.0       2
15.0      1
14.0      1
10.0      1
Name: visualizaciones, dtype: int64

# Armado de los sets de test y de entrenamiento

In [9]:
from sklearn.model_selection import train_test_split
set_entrenamiento = get_datos_entrenamiento(50000)
train, test = train_test_split(set_entrenamiento, test_size=0.2)


In [11]:
train['sepostulo'].value_counts()

True     36206
False    21622
Name: sepostulo, dtype: int64

In [12]:
test['sepostulo'].value_counts()

True     9082
False    5375
Name: sepostulo, dtype: int64

In [13]:
test.head()

,edad_postulante,genero_postulante,maximo_nivel_educativo_postulante,nivel_laboral,titulo,sepostulo
7872,57.0,0.0,48.0,3.0,Responsable Administrativo,True
20894,36.0,1.0,18.0,3.0,Supervisor de Expedición Turnos Rotativos,True
6897,58.0,1.0,48.0,3.0,Responsable Administrativo,True
15472,31.0,1.0,44.0,1.0,Data Entry Administrativo,False
15594,33.0,0.0,44.0,2.0,Analista de RRHH Ssr,True


In [9]:
set_test.shape

(100000, 9)

# Entrenamiento

In [111]:
from sklearn.model_selection import train_test_split
from sklearn import svm

In [ ]:
def get_predictor(train):
    # Prepare feature and dependent variable along with categorical encoding
    X=pd.get_dummies(train.loc[:, x_entrenamiento()])
    y=train.loc[:, 'sepostulo']
    clf = svm.SVC()
    return clf.fit(X, y)

predictor = get_predictor(set_entrenamiento)

# Prueba y resultados para la competencia

In [20]:
def obtener_predicciones(predictor, set_test):
    return predictor.predict(set_test[x_entrenamiento()]).astype(int)

def guardar_resultados(fileName, predictor, set_test):
    result = obtener_predicciones(predictor, set_test)
    set_test['sepostulo'] = result
    set_test[['id','sepostulo']].set_index('id').to_csv(fileName)
    return

guardar_resultados('100K ultimo.csv', predictor, set_test)
set_test['sepostulo'].value_counts()

1    64627
0    35373
Name: sepostulo, dtype: int64